**Содержание**<a id='toc0_'></a>    
- [Обработка ошибок](#toc1_)    
  - [Логические ошибки и исключительные ситуации](#toc1_1_)    
  - [Выявление логических ошибок на этапе разработки](#toc1_2_)    
  - [Способы сообщения об ошибке](#toc1_3_)    
  - [Исключения](#toc1_4_)    
  - [Stack unwinding](#toc1_5_)    
  - [Почему не стоит бросать встроенные типы?](#toc1_6_)    
  - [Стандартные классы исключений](#toc1_7_)    
  - [Исключения в стандартной библиотеке](#toc1_8_)    
  - [Как обрабатывать ошибки?](#toc1_9_)    
  - [Исключения в деструкторах и конструкторах](#toc1_10_)    
    - [Исключения в деструкторах](#toc1_10_1_)    
    - [Исключения в конструкторе](#toc1_10_2_)    
    - [Исключения в списке инициализации](#toc1_10_3_)    
- [Спецификация исключений](#toc2_)    
  - [Спецификация исключений](#toc2_1_)    
  - [Ключевое слово noexcept](#toc2_2_)    
  - [noexcept для деструктора](#toc2_3_)    
  - [Использование noexcept](#toc2_4_)    
  - [Условный noexcept](#toc2_5_)    
  - [Зависимость от noexcept](#toc2_6_)    
- [Гарантии безопасности исключений](#toc3_)    
  - [Гарантии безопасности исключений](#toc3_1_)    
  - [Строгая гарантия безопасности исключений](#toc3_2_)    
  - [Как добиться строгой гарантии?](#toc3_3_)    
  - [Как добиться строгой гарантии: вручную](#toc3_4_)    
  - [Как добиться строгой гарантии: RAII](#toc3_5_)    
  - [Как добиться строгой гарантии: swap](#toc3_6_)    
  - [Проектирование с учётом исключений](#toc3_7_)    
  - [Проектирование с учётом исключений (продолжение)](#toc3_8_)    
  - [Использование unique_ptr](#toc3_9_)    
  - [Заключение](#toc3_10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Обработка ошибок](#toc0_)

## <a id='toc1_1_'></a>[Логические ошибки и исключительные ситуации](#toc0_)

**Логические ошибки.**

Ошибки в логике работы программы, которые происходят из-за неправильно написанного кода, т.е. это ошибки программиста:

- выход за границу массива,
- попытка деления на ноль,
- обращение по нулевому указателю,
- . . .

**Исключительные ситуации.**

Ситуации, которые требуют особой обработки. Возникновение таких ситуаций — это „нормальное“ поведение программы.

- ошибка записи на диск,
- недоступность сервера,
- неправильный формат файла,
- . . .

## <a id='toc1_2_'></a>[Выявление логических ошибок на этапе разработки](#toc0_)

**Оператор** static_assert (`static` == проверки на этапе компиляции)

    #include<type_traits>   // is_integral is_signed - на этапе компиляции

    // #defunt NDEBUG          // все ассерты рантайма превращаются в nop
    #include <cassert>      // assert (макрос) - не этапе выполнения
    
    template<class T>
    void countdown(T start) {
        static_assert(std::is_integral<T>::value && std::is_signed<T>::value,
                      "Requires signed integral type");
        
        assert(start >=0);

        while (start >= 0) {
            std::cout << start-- << std::endl;
        }
    }

макрос `assert`

    #ifdef NDEBUG
    #  define assert(condition) ((void)0)
    #else
    #  define assert(condition) /*implementation defined*/
    #endif

Если очень хочется сообщение об ошибке, то можно добавить строку к условию.
    
    assert(x > 0 && "Message")

**Тест**

Выберите типы с которым следующий static_assert не вызовет ошибку компиляции.

    static_assert( std::is_arithmetic<T>::value || 
                (std::is_pointer<T>::value && 
                    std::is_class<typename std::remove_pointer<T>::type>::value), 
            "T can not be used here" );

- T должен быть или числовым (int/float) или указателем на класс


**да** int / std::string *

**нет** int * / std::vector<int> / std::vector<int*> / std::string



## <a id='toc1_3_'></a>[Способы сообщения об ошибке](#toc0_)

Пусть есть функция:

    size_t write(string file, string data);

Сообщить об ошибке можно так:

**Возврат статуса операции** - была ли ошибка (перепеписать функцию или обернуть):
    
    bool write(string file, string data, size_t & bytes);

**Возврат кода ошибки** - аналогично + заводятся константы/перечисление кодов ошибок:

    int const OK = 0, IO_WRITE_FAIL = 1, IO_OPEN_FAIL = 2;
    int write(string file, string data, size_t & bytes);


**Глобальная переменная** для кода ошибки - сигнатуру функции менять не потребуется, но функцию менять все равно придется (так делается в С-style) - проверяем глобальную переменную, в которую должна писать `write`:

    size_t write(string file, string data);

    size_t bytes = write(f, data);
    if (errno) {
        cerr << strerror(errno);
        errno = 0;
    }

*) глобальная переменная - это хуже изменения сигнатуры, т.к. код становится немасштабируемым, его нельзя будет использовать в многопоточных делах. 

**Исключения** - не имеют перечисленных выше сложностей

Замечания:

В защиту `errno` можно отметить, что это не просто глобальная переменная. Часто это макрос, реализующий вызов соответствующей функции, которая реализует доступ к переменной, локальной в смысле потока. Т.е. errno является потокобезопасной, по крайней мере, в POSIX-системах errno своя для каждого потока.

В случае классов есть ещё комбинированный вариант - возврат `bool`, а код ошибки класс выдаст по запросу отдельным методом, если он вообще требуется. Такого подхода придерживаются многие C++ библиотеки, самый очевидный пример - Qt, которая ничуть не страдает из-за отсутствия исключений.



## <a id='toc1_4_'></a>[Исключения](#toc0_)

Исключение - это объект с информацией об ошибке

    size_t write(string file, string data) {
        if (!open(file)) throw FileOpenError(file);
        //...
    }

    double safediv(int x, int y) {
        if (y == 0) throw MathError("Division by zero");
        return double(x) / y;
    }

    void write_x_div_y(string file, int x, int y) {
        try {
            write(file, to_string(safediv(x, y)));
        } catch (MathError & s) {
            // обработка ошибки в safediv
        } catch (FileError & e) {
            // обработка ошибки в write
        } catch (...) {
            // все остальные ошибки (доступа к объекту исключения тут не будет, только факт исключения)
        }
    }

*) `FileOpenError` потомок `FileError` и исключения могут обрабатываться по ссылке на базовый класс

## <a id='toc1_5_'></a>[Stack unwinding](#toc0_)

*) рекомендуют переводить как "сворачивание стека", т.к. распространенные варианты "раскрутка стека"/"разворачивание стека" и т. п. соответствуют неправильной метафоре — на самом деле стек именно "сворачивается", т. е. уменьшается, а не "раскручивается", т. е. увеличивается.

При возникновении исключения объекты на стеке уничтожаются в естественном (обратном) порядке.

    void foo() {
        D d;
        E e(d);
        if (!e) 
            throw F(); // объект F() копируется в спец.буфер для хранения исключений *
        G g(e);
    }

    void bar() {
        A a;
        try {
            B b;
            foo();
            C c;
        } catch (F & f) {
            // обработка и пересылка
            throw f; // тонкость
        }
    }

*) стандарт не определяет как должен быть устроен буфер исключений, зависит от компиляторв

Если брошено исключени в `foo()` локальные переменные удаляются в обр.порядке (Stack unwinding - разворачивание стека):
- удаляется `E e(d)` в `foo`
- удаляется `D d` в `foo`
- выход из `foo();` в `bar`
- удаляется `B b` в `bar`
- дошли до `try` - конец разворачивания стека, `A a` в `bar` останется на стеке

В это случае `C c` и `bar` и `D d` в `foo` не будут созданы.

Тонкость. Вместо

    throw f;

можно (в большинстве случаев лучше) писать просто

    throw;

тогда будет переслан тот же самый объект класса `F`. Если в блоке catch мы ловим exception f базового класса, то при использовании `throw f` мы делаем копию уже от базового класса и выше по иерархии мы теряем информацию о том исключении, которое было брошено изначально. Так что если нам не нужно бросить новое исключение, а эскалировать текущее, использовать лучше только `throw` без параметров.

## <a id='toc1_6_'></a>[Почему не стоит бросать встроенные типы?](#toc0_)

Так делать можно, но не нужно:

    int foo() {
        if (...) throw -1;
        if (...) throw 3.1415;
    }

    void bar(int a) {
        if (a == 0) throw string("Not my fault!");
    }

Потому что придется ловить именно конкретно этот тип, причем информации об исключении будет крайне мало:

    int main () {
        try { bar(foo());
        } catch (string & s) {
            // только текст
        } catch (int a) {
            // мало информации
        } catch (double d) {
            // мало информации
        } catch (...) {
            // нет информации
        }
    }

**Q**. Как происходит сопоставление типа бросаемого объекта и типа в catch в райнтейме, когда типов уже нет как таковых (кроме классов с вирт.методами)? 

**А**. Зависит от компилятора. В gcc, например, вместе с объектом исключения передаётся и type_info, по которому сопоставляется catch.

## <a id='toc1_7_'></a>[Стандартные классы исключений](#toc0_)

Базовый класс для всех исключений (в `<exception>`):

    struct exception {
        virtual ~exception();
        virtual const char* what() const;       // текстовое представление ошибки
    };

Стандартные классы ошибок (в `<stdexcept>`):

- logic_error: 
    - производные domain_error, invalid_argument, length_error, out_of_range
- runtime_error: 
    - производные range_error, overflow_error, underflow_error

Например, чтобы отловить все исключения, унаследованные от `exception`

    int main() {
        try { ... }
        catch (std::exception const& e) {
            std::cerr << e.what() << ’\n’;
        }
    }

*) `catch(...)` по сути нужен только если мы не хотим падения программы с unhandled exception, когда стек вызовов уже развернут до main, а подходящего обработчика так и не нашлось

## <a id='toc1_8_'></a>[Исключения в стандартной библиотеке](#toc0_)

Метод `at` контейнеров array, vector, deque, basic_string,
bitset, map, unordered_map бросает `out_of_range`.

Оператор `new T` бросает `bad_alloc`.

Оператор `new (std::nothrow) T` в возвращает `0`.

Оператор `typeid` от разыменованного нулевого указателя бросает `bad_typeid`.

Потоки ввода-вывода по умолчанию **не бросают** никаких исключений, только вручную.

    std::ifstream file;

    // в объект потока маски исключений добавляются так
    file.exceptions( std::ifstream::failbit | std::ifstream::badbit ); // Logical error on i/o | Read/writing error on i/o 

    try {
        file.open ("test.txt");
        cout << file.get() << endl;
        file.close();
    }
    
    // тип исключения в потоках определяется так
    catch (std::ifstream::failure const& e) {
        cerr << e.what() << endl;
    }

*) У класса ios_base, и как следствие, у всех порождённых от него классов для работы с потоками ввода/вывода, есть четыре флага состояния: goodbit, badbit, eofbit, failbit. Метод exceptions(iostate mask) устанавливает битовую маску, для которой поток будет кидать исключения, если любой из указанных битов состояния был установлен вследствие ошибки. В данном примере исключение будет кидаться, если badbit или failbit установились в состояние true.

## <a id='toc1_9_'></a>[Как обрабатывать ошибки?](#toc0_)

Есть несколько „правил хорошего тона“.

- Разделяйте „ошибки программиста“ и „исключительные ситуации“.
- Используйте assert и static_assert для выявления ошибок на этапе разработки.
- В пределах одной логической части кода обрабатывайте ошибки централизованно и однообразно.
- Обрабатывайте ошибки там, где их можно обработать.
- Если в данном месте ошибку не обработать, то пересылайте её выше при помощи исключения.
    - пересылать просто `throw`, чтобы не было копирования и не терялась информация об исходном классе исключения, если оно принималось по ссылке на базовый класс
- Бросайте только стандартные классы исключений или производные от них.
- Бросайте исключения по значению, а отлавливайте по ссылке 
    - не будет копирования, при котром тоже м.б. исключение
- Отлавливайте все исключения в точке входа.
    - это последний шанс поймать исключение, которое ранее нигде не поймалось



**Задача**

В стандартной библиотеке есть семейство функций to_string для преобразования чисел в строки. Однако обратное преобразование не такое удобное — для каждого числового типа есть своя функция (например, strtoi для int).
В данном задании вам предлагается написать шаблонную функцию from_string, которая умеет преобразовывать строку в разные типы. Для реализации from_string предлагается воспользоваться классом std::istringstream, который представляет собой поток ввода из строки, т.е. для преобразования строки в тип T предлагается прочитать значение типа T из потока при помощи оператора >>. В случае неудачного преобразования функция должна бросать исключение bad_from_string, класс которого вам нужно реализовать самостоятельно.

Указания

- Для того, чтобы учитывать пробельные символы, используйте std::noskipws (например, если строка с числом начинается с пробела или заканчивается пробелом, то это должно быть ошибкой).
- При переопределении метода what класса std::exception после сигнатуры метода нужно указывать ключевое слово noexcept (про него будет рассказано позже).
- Помните, что считывание std::string из потока означает считывание одного слова (т.е. без пробельных символов), а считывание char — одного символа.
- Флаг eof() у потоков устанавливается только, если не удалось прочитать символ: если при чтении из потока с 5-ю символами прочли 5 символов, но при этом 6-ой (отсутствующий) символ прочесть не пытались, то eof() будет выдавать false.
- Если Вы определили исключение с ключевым словом class, но не забудьте, что нужно унаследоваться от std::exception с ключевым словом public.
- Не забудьте определить конструктор bad_from_string от char const * или от std::string.

    #include <string>
    #include <sstream>      // istringstream
    #include <exception>
    
    // описание класса исключения bad_from_string
    class bad_from_string : public std::exception
    {
        const std::string info;
    public:
        bad_from_string(const std::string e ="from_string exception") : info(e){}
        const char* what() noexcept // noexcept обязательно
        {
            return info.c_str();
        }
    };

    // функция from_string
    template<class Type>
    Type from_string(std::string const& s)
    {
        std::istringstream stream(s);

        Type res;
        
        // std::noskipws не пропускать ' ': строка c пробелами - это ошибка
        stream >> std::noskipws >> res;
        std::string msg = "error converting to ";

        if (stream.fail()) 
            throw bad_from_string(msg + typeid(Type).name());
        stream.get(); 

        if(stream.good()) // все ОК, а должен быть EOF!
            throw bad_from_string(msg + typeid(Type).name() + ": string has ws chars");

        return res;
    }


## <a id='toc1_10_'></a>[Исключения в деструкторах и конструкторах](#toc0_)

### <a id='toc1_10_1_'></a>[Исключения в деструкторах](#toc0_)

Исключения могут использовать в деструкторах и конструкторах как и везде, но!

**Исключения не должны покидать деструкторы.** (это ответственность разработчика)

Двойное исключение:

    void foo() {
        try {
            Bad b; // исключение в деструкторе
            bar(); // исключение
        } catch (std::exception & e) {
            // ...
        }
    }

- Пусть `Bad b` имеет деструктор, который может бросать исключения. Тогда срабатывание исключения в `bar()` вызывает сворачивание стека и срабатывание деструктора `b`, т.е. во время возникновения одного исключения может возникнуть еще одно (если в деструкторе тоже сработает исключение). С++ предполагает, что одновременно может возникать только 1 исключение, поэтому в таком случае программа гарантированно завершится по сис.вызову `terminate` (без вариантов)

Неопределённое поведение:

    void bar() {
        Bad * bad = new Bad[100];
        // ислючение в деструкторе №20
        delete [] bad;
    }

- пусть есть массив объектов в деструкторе элементов которого может бросаться исключение. Если при удалении такого массива в некотором элементе сработает исключение, то удаление массива остановится, исключение будет распространяться по стеку (может где-то и отловится), но суть в том, что массив останется висеть в памяти и не освободит ее, и в нем будут как разрушенные элементы, так и не разрушенные

**Тест**

Какие из этих конструкций можно указывать в деструкторе, не оборачивая их в try-catch блок.

    // 1 исключение нужно перехватить
    throw std::runtime_error("Some message");

    // 2 может закончиться память std::bad_alloc
    int * data = new int[size];

    // 3 ОК, std::bad_alloc невозможен
    int * data = new (std::nothrow) int[size];

    // 4 ОК, предполагается что деструктор корректный 
    delete [] data;

    // 5 s копируется, а вдруг не хватит памяти? std::bad_alloc, а вдруг используется ленивое копирование и самой строки уже нет 
    // data_ - это поле класса типа std::vector<string>
    for (auto s : data_) { len += s.size(); }

    // 6 ОК
    // data_ - это поле класса типа std::vector<string>
    size_t len = 0;
    for (auto & s : data_) { len += s.size(); }

**3, 4, 6 можно не оборачивать**

### <a id='toc1_10_2_'></a>[Исключения в конструкторе](#toc0_)

В отличие от деструктора покидание исключением конструктора это НОРМА, и даже желательно.

Исключения — это единственный способ прервать конструирование объекта и сообщить об ошибке. Можно было бы прервать создание ретурном, но как тогда понять, что это ошибочный объект, он все равно создастся.

    struct Database {
        explicit Database(string const& uri) {
            if (!connect(uri))
                throw ConnectionError(uri);
        }
        ~Database() { disconnect(); }
        // ...
    };

Предположим БД создается в куче, даже в этом случае сворачивание стека будет корректно отработано уничтожение объекта, это так задумано. Деструктор вызываться не будет (он нужен только для корректно созданных объектов по смыслу)

    int main() {
        try {
            Database * db = new Database("db.local");
            db->dump("db-local-dump.sql");
            delete db;
        } catch (std::exception const& e) {
            std::cerr << e.what() << ’\n’;
        }
    }

Замечание:

- а если память в куче выделяется в конструкторе и потом бросается исключение? 
    - тогда память утечет, никто же не знает заранее как устроен код конструктора
    - если надо бросать такие исключения, то можно там же в конструкторе их и ловить и обрабатывать (catch-rethrow идиома), но это уже экзотика и применительно к конструкторам не совсем то как он по идее должен использоваться


### <a id='toc1_10_3_'></a>[Исключения в списке инициализации](#toc0_)

**Поля класса создаются перед входом в тело конструктора**. Даже если весь код конструктора завернуть в трайкеч исключения при создании объектов списка инициализации не поймаются. 

Для этого сделали специальный синтаксис трайкеча: `try` перед списком инициализации и `catch` после определения конструктора

Позволяет отловить исключения при создании полей класса.

    struct System
    {
        System(string const& uri, string const& data)
        try : db_(uri), dh_(data)
        {
            // тело конструктора
        }
        catch (std::exception & e) {
            log("System constructor: ", e);
            throw; // пробрасываем наверх
        }
    
        Database db_;
        DataHolder dh_;
    };

*) если не пробросить исключение из такой конструкции, компилятор сделает это сам, ибо объект то в итоге не создался, полюбому исклоючение надо где-то обрабатывать

**Тест**

Давайте изучим следующий код (предполагается, что конструкторы классов A и B не бросают исключения, если не указано обратное):

    extern bool X;
    extern bool Y;

    struct A
    {
        A() { ... }

        ~A() 
        {
            try 
            {
                bar();
            } 
            catch (std::exception const & e) 
            {
                log("~A()", e.what());
            }
        }
        B b;
    };

    void foo() 
    {
        A a;
        if (X)
            throw std::runtime_error("Some message");
    }

    void bar() 
    {
        if (Y)
            throw std::logic_error("Some message");
    }

Если при вызове foo() X = false, то и никакое исключение не покинет функцию foo().

Если при вызове foo() X = true, то функцию foo() покинет только исключение std::runtime_error.

Никакие двойные исключения тут невозможны

**Тест**

Давайте изучим следующий код:

    #include <stdexcept>

    extern bool X;

    struct MyError : std::runtime_error
    {
        MyError(char const* s) 
            : std::runtime_error(s)
        {
            if (X)
                throw std::logic_error("Logic error");
        }
    };

    void foo()
    {
        throw MyError("My error");
    }
Если Х, то MyError создан не будет, и бросать нечего

**ДА**
- Если при вызове foo() X = true, то за пределы функции foo будет выброшено исключение std::logic_error.
- Если при вызове foo() X = false, то за пределы функции foo будет выброшено исключение MyError.
- Если при вызове foo() X = true, то будет вызван деструктор std::runtime_error.

**НЕТ**
- Если при вызове foo() X = true, то произойдёт аварийное завершение программы.
- Если при вызове foo() X = true, то за пределы функции foo будет выброшено исключение MyError.
- Если при вызове foo() X = true, то будет вызван деструктор MyError.

# <a id='toc2_'></a>[Спецификация исключений](#toc0_)


## <a id='toc2_1_'></a>[Спецификация исключений](#toc0_)

Устаревшая возможность C++, позволяющая указать список исключений, которые могут быть выброшены из функции. Но в отличие от Java С++ не делает проверки времени компиляции на то, что функции бросают именно эти исключения. Вместо этого делается проверка времени выполнения

    void foo() throw(std::logic_error) {
        if (...) throw std::logic_error();
        if (...) throw std::runtime_error();
    }

В примере выше, если сработает второй if с непредусмотренным исключением, то программа аварийно завершится. Делается это следущим образом, при компиляции код заменяется на примерно такой:

    void foo() {
        try {
            if (...) throw std::logic_error();
            if (...) throw std::runtime_error();
        } catch (std::logic_error & e) {
            throw e;
        } catch (...) {
            terminate();
        }
    }

Если сработал второй if, то вызывается terminate()

**Короче, не очень удобный механизм, единственное актуальное применение, это указывать пустой список с расчетом на то, что функция не должна бросать исключения, а если бросила, то это аварийное завершение.**

    void foo() throw() {...};

- т.е. гарантируется остановка именно в той функции, которая нарушила промис о том, что не должна кидать исключения

## <a id='toc2_2_'></a>[Ключевое слово noexcept](#toc0_)

Вместо этого в С++11 появилось новое ключевое слово `noexept`

Используется в двух значениях/контекстах:
- Спецификатор функции, которая не бросает исключение.
- Оператор, проверяющий во время компиляции, что выражение специфицированно как небросающее исключение.

Если функцию со спецификацией `noexcept` покинет исключение, то стек **не обязательно** будет свёрнут, перед тем как программа завершится. В отличие от аналогичной ситуации с `throw()`.

- Использование спецификации `noexcept` позволяет компилятору лучше оптимизировать код, т.к. не нужно заботиться о сворачивании стека.
- Если исключение попытается покинуть `noexcept` метод, то вызывается `std::terminate`

*) noexcept не проверяет функцию на наличие явных операторов throw, но компилятор может давать предупреждения 

**Тест**

Какие методы класса String нужно (можно) специфицировать как noexcept?

    Default constructor: 1
    Copy constructor:    0
    Move constructor:    1
    op =:                0
    Move op =:           1
    size():              1
    clear():             1
    delete:              1

В общем, там, где память может выделятся - нет, возвращается - да, можно noexcept
- можно реализовать конструктор по-умолчанию без выделения памяти, а лишь просто проинициализировать данные объекта, т.е. таким образом не будет исключений в таком конструкторе

## <a id='toc2_3_'></a>[noexcept для деструктора](#toc0_)

Отдельно стоит упомянуть, что компилятор может (с C++11 обязан) неявно специфицировать noexcept для деструктора, если вы не сделали это самостоятельно.

Например, при выполнении следующего кода программа завершится аварийно после вызова std::terminate, т.к. компилятор неявно специфицирует деструктор  класса A как noexcept.

    struct A
    {
        ~A() { throw std::logic_error("Exception is leaving destructor"); }
    };


    int main()
    {
        try
        {
            A a;
        }
        catch (std::exception const& e)
        {
            std::cout << e.what() << std::endl;
        }

        return 0;    
    }

## <a id='toc2_4_'></a>[Использование noexcept](#toc0_)

Пусть есть две функции, одна `noexcept`

    void no_throw() noexcept;
    void may_throw();

Или если структура состоит только из типов, у которых конструкторы `noexcept`, то ее конструктор будет тоже `noexcept`

    // копирующий конструктор noexcept
    struct NoThrow { int m[100] = {}; };

А тут без `noexcept`, точнее `noexcept(false)`

    // копирующий конструктор noexcept(false)
    struct MayThrow { std::vector<int> v; };

Тогда информацию о спецификации noexcept этих типов и выражений от них можно проверять на этапе компиляции

    MayThrow mt;
    NoThrow nt;
    
    bool a = noexcept(may_throw()); // false
    bool b = noexcept(no_throw()); // true
    
    bool c = noexcept(MayThrow(mt)); // false
    bool d = noexcept(NoThrow(nt)); // true

## <a id='toc2_5_'></a>[Условный noexcept](#toc0_)

В спецификации noexcept можно использовать условные выражения времени компиляции. Функция объявляется компилятором noexcept только если выражение true, иначе объявляется noexcept(false)

Например, фунция поэлементного свопа 2 массивов будет компилироваться как `noexcept` если операция свопа их первых элементов `noexcept(true)`:

    template <class T, size_t N>
    void swap(T (&a)[N], T (&b)[N]) // *
                noexcept(noexcept(swap(*a, *b)));

*) значение в swap передаются явно по ссылке, хотя массивы и так по умолчанию передаются по ссылке. Они неявно будут преобразованы в указатели. Это видимо сделано так для того, чтобы в функцию можно было передать только реальный массив причем размера N, а не просто приведенный к типу указатель...


Или, функция свопа для аналога `std::pair` будет компилироваться как `noexcept` если операция свопа их первых элементов и вторых элементов будет `noexcept(true)`

    template <class T1, class T2>
    struct pair {
        void swap(pair & p)
            noexcept(noexcept(swap(first, p.first)) &&
                     noexcept(swap(second, p.second)))
        {
            swap(first, p.first);
            swap(second, p.second);
        }
        
        T1 first;
        T2 second;
    };


**Задача**

Выше мы воспользовались тем, что функция swap имеет аргументы нужных нам типов. В тех случаях, когда аргументов нужного типа у функции нет, можно использовать std::declval<T>(), который позволяет "определить" значение типа T в различных контекстах, в т.ч. в noexcept.

Специфицируйте noexcept для шаблонной функции do_math, если известно, что в её реализации значения типа T копируются,  присваиваются и складываются при помощи оператора +.

Для простоты будем считать, что перемещающих методов у типа T нет.

Пример:

    bool b1 = noexcept(do_math<int>()); // true

    bool b2 = noexcept(do_math<std::string>()); // false

Hint: в данном задании вполне можно обойтись std::declval<T>(), но при желании можно так же заглянуть в библиотеку поддержки типов и найти там подходящую проверку.

    #include <utility> // std::declval

    // внутри do_math объекты типа T
    // - копируются
    // - присваиваются
    // - складываются оператором +
    template<class T>
    void do_math() 
        noexcept(noexcept(std::declval<T>() + std::declval<T>()) 
                 && std::is_nothrow_copy_constructible<T>::value
                 && std::is_nothrow_assignable<T&, T>::value)
    {}

**Задача**

Решите предыдущую задачу в предположении, что у класса T могут быть перемещающие методы.

    #include <utility> // std::declval

    // внутри do_math объекты типа T
    // - копируются
    // - присваиваются
    // - перемещаются
    // - складываются оператором +
    template<class T>
    void do_math() 
        noexcept(noexcept(std::declval<T>() + std::declval<T>()) 
                && std::is_nothrow_copy_constructible<T>::value
                && std::is_nothrow_assignable<T&, T>::value
                && std::is_nothrow_move_assignable<T&>::value
                && std::is_nothrow_move_constructible<T>::value)
    {}

*) `...assignable` проверялки должны принимать по ссылке аргумент которому присваивается, чтобы не раскрывалось при разборе в, например, для интов `1 = 1`, т.е. должно быть lvalue

## <a id='toc2_6_'></a>[Зависимость от noexcept](#toc0_)

Проверка `noexcept` используется в стандартной библиотеке для обеспечения **строгой гарантии безопасности исключений** с помощью `std::move_if_noexcept` (например, в `vector::push_back` новый объект создается перемещаютщим конструктором, если он noexcept, иначе - копирующим конструктором, что дольше).

    struct Bad {
        Bad() {}
        Bad(Bad&&); // может бросить
        Bad(const Bad&); // не важно
    };

    struct Good {
        Good() {}
        Good(Good&&) noexcept; // не бросает
        Good(const Good&) ; // не важно
    };

    Good g1;
    Bad  b1;

    Good g2 = std::move_if_noexcept(g1); // move
    Bad  b2 = std::move_if_noexcept(b1); // copy

**Зачем?**

Смысл в том, что копирующее создание обращаемо, т.е. если произошло исключение, то останется исходная валидная копия объекта - это и называется строгая гарантия безопасности исключений. Исключение при перемещении приведет к тому, что объект как минимум инвалидируется.

# <a id='toc3_'></a>[Гарантии безопасности исключений](#toc0_)

## <a id='toc3_1_'></a>[Гарантии безопасности исключений](#toc0_)

**Гарантия отсутствия исключений**

“Ни при каких обстоятельствах функция не будет генерировать исключения”.
- главное условие - никакие исключения не покидают функцию
- исключения если и происходят, то внутри функции они все и обрабатываются
- именно такие функции обычно помечают noexcept

**Базовая гарантия**

“При возникновении любого исключения состояние программы останется согласованным”.
- должно обеспечиваться для всех функций программы

**Строгая гарантия** 

“Если при выполнении операции возникнет исключение, то программа останется том же в состоянии, которое было до начала выполнения операции”.
- обеспечивается транзакционность, если началась некоторая операция и произошло исключение, то "транзакция" откатывается к предыдущему состоянию

## <a id='toc3_2_'></a>[Строгая гарантия безопасности исключений](#toc0_)

В каком случае мы не можем обеспечить строгую гарантию безопасности исключений?
- При наличии взаимодействия со внешним окружением, т.е. необратимых действий (вроде отправки данных по сети и т.п.)

Как обеспечить строгую гарантию безопасности исключений в остальных случаях?
- Выполнять операцию над копией состояния программы.
- Если операция прошла успешно, заменить состояние на копию.

Когда можно обеспечить строгую гарантию эффективно?
- По смыслу придется много копировать, соответственно, нужно минимизировать число операций копирования
- Это вопрос архитектуры приложения.

**Примеры**

Когда **не получится обеспечить строгую гарантию безопасности исключений**

- Передача данных по USB.
- Передача файлов по Bluetooth
- Копирование массива объектов типа std::string в std::ostream_iterator (может передавать данные куда угодно, на экран, в сеть и т.д.).

Когда можно обеспечить:
- Преобразование XML данных в оперативной памяти.
- Копирование массива объектов типа std::string в std::list.
- Сортировка массива объектов типа std::string (если мы изменили порядок элементов в массиве, как мы можем вернуть первоначальный порядок? сортировать не массив, а его копию).

## <a id='toc3_3_'></a>[Как добиться строгой гарантии?](#toc0_)

Пусть есть класс массива и метод изменения его размера `resize`

    template<class T>
    struct Array
    {
        void resize(size_t n)
        {
            T * ndata = new T[n];
            for (size_t i = 0; i != n && i != size_; ++i) // до min(n,size_)
                ndata[i] = data_[i];
                
            delete [] data_;
            data_ = ndata;
            size_ = n;
        }

        T * data_;
        size_t size_;
    };

Проблемные места:
1. `new` может бросить `bad_alloc`
    - с т.з. безопасности исключений это не проблема, т.к. объект `Array` еще не менялся
2. класс шаблонный и у типа `T` может быть конструктрор, который бросает исключения
    - исключение может вылететь из любого из `n` конструкторов в `new`
        - тоже не проблема, копилятор гаратирует, что при исключениях в конструкторах в `new` память будет освобождена, а данные `Array` опять же не менялись
3. присваивание `ndata[i] = data_[i]`
    - если будет исключение, то оно выйдет наружу, а указатель `T * ndata` останется не освобожденным (утечка памяти)

Данный код не удовл. даже базовым гарантиям безопасности исключений, т.к. утечка памяти - это **несогласованное состояние программы**

## <a id='toc3_4_'></a>[Как добиться строгой гарантии: вручную](#toc0_)

Добавление трайкеч и обработка исключений

    template<class T>
    struct Array
    {
        void resize(size_t n) {
            T * ndata = new T[n];
            try {
                for (size_t i = 0; i != n && i != size_; ++i)
                    ndata[i] = data_[i];
            } catch (...) {
                delete [] ndata;
                throw;
            }
            delete [] data_;
            data_ = ndata;
            size_ = n;
        }

        T * data_;
        size_t size_;
    };

1. `new` обрабатывать не надо, более того, не надо вкладывать его в трайкеч, т.к. возможна ситуация, когда освобождение памяти (деструкторы) будет вызываться для несозданных элементов.
2. присваивание, если вызовет исключение, то мы освободим выделенную память

Утечки памяти нет, состояние массива не изменилось

## <a id='toc3_5_'></a>[Как добиться строгой гарантии: RAII](#toc0_)

Если у нас есть ресурс, который требует освобождения в случае исключений (например, память),то вместо отлавливания исключений вручную можно создать RAII-объект
- RAII (Resource Acquisition Is Initialization) - идеома, когда получение некоторого ресурса неразрывно совмещается с инициализацией, а освобождение — с уничтожением объекта. Обычно реализуется передачей ресурса некоторому объекту на стеке, который получает ресурс в своем конструкторе и освобождает в деструкторе.

Тут ресурс - это указатель, поэтому на помощь спешит умный указатель из STL. У него есть специализация для работы с массивами.


    template<class T>
    struct Array
    {
        void resize(size_t n) {
            unique_ptr<T[]> ndata(new T[n]);
            
            for (size_t i = 0; i != n && i != size_; ++i)
                ndata[i] = data_[i];

            data_ = std::move(ndata);
            size_ = n;
        }

        unique_ptr<T[]> data_;
        size_t size_;
    };

Не только будет хранить в нем новый массив в `resize`, но и данные самого объекта. 

Если в присваивании возникнет исключение, то начнется сворачивание стека и деструктор локального `unique_ptr` освободит указатель. Исходное состояние объекта при этом не пострадает

## <a id='toc3_6_'></a>[Как добиться строгой гарантии: swap](#toc0_)

Для консервативных операций (где нет общения с "внешним миром") почти всегда доступен вариант с копированием данных программы и выполеннием операции над копией данных. Если возникнет исключение, то данные исх.объекта не пострадают.

    template<class T>
    struct Array
    {
        void resize(size_t n) {
            Array t(n);
            
            for (size_t i = 0; i != n && i != size_; ++i)
                t[i] = data_[i];    // какбэ для класса Array определен оператор [ ]
            t.swap(*this);          // или swap(t) - без разницы
        }

        T * data_;
        size_t size_;
    };

Создаем новый объект нужного размера, копируем в него объекты и есть все ОК, меняем местами с текущим объектом.

Здесь используется тот факт, что операция `swap` обычно реализуется как `noexcept`

**Тест**

Какие из следующих функций обеспечивают строгую гарантию безопасности исключений?

    // НЕТ порядок не определен и если в new V исключение, то new T - утечка
    std::pair<T*,V*> fun1() 
    {
        return std::make_pair(new T, new V);
    }

    // НЕТ, порядок не определен и new T может упасть до того, как new V будет помещён в std::unique_ptr<T>()
    std::pair<std::unique_ptr<T>,std::unique_ptr<V>> fun2() 
    {
        return std::make_pair(
                    std::unique_ptr<T>(new T),
                    std::unique_ptr<V>(new V));
    }

    std::pair<std::unique_ptr<T>,std::unique_ptr<V>> fun3() 
    {
        std::unique_ptr<T> pt(new T);
        std::unique_ptr<V> pv(new V);
        return std::make_pair(std::move(pt), std::move(pv));
    }

    void fun4(std::vector<std::string> & v)
    {
        std::reverse(v.begin(), v.end());
    }
    
    // НЕТ, в fill идет присваивание, что потенциально может вызывать выделение памяти
    void fun5(std::vector<std::string> & v, std::string const& s)
    {
        std::fill(v.begin(), v.end(), s);
    }

    void fun6(std::vector<std::string> & v, std::string const& s)
    {
        std::vector<std::string>(v.size(), s).swap(v);
    }

- смотрим возможную [реализацию](https://en.cppreference.com/)
    - swap noexcept
    - reverse использует iter_swap, который использует swap
- где new в аргументах, порядок вычисления не определен, поэтому если второй кинет исключение, то первый не будет освобожден при сворачивании стека и останется аллоцированным (утечка)
    - порядок вычисления НАСТОЛЬКО неопределен, что в `fun2` нет гарантий, что внутренние скобки предшествуют только внешним!!! Может так случиться, что сначала будет вызван один new, а потому другой new и только после этого создадутся unique_ptr, такой порядок допустим стандартом
    - тут некая тонкость - на стек помещается вызов одного аргумента, если он отработал ОК, то он со стека снимается, а результат пишется в аргументы текущего фрейма. Поэтому при сворачивании стека к нему уже не будет доступа.

## <a id='toc3_7_'></a>[Проектирование с учётом исключений](#toc0_)

Роль архитектуры.

Рассмотрим традиционный интерфейс стека:
    
    template<class T>
    struct Stack
    {
        void push(T const& t)
        {
            data_.push_back(t);
        }

        T pop()
        {
            T tmp = data_.back();
            data_.pop_back();
            return tmp;
        }
        
        std::vector<T> data_;
    };

1. Метод `push` обесп.строг.гар.безоп.искл., т.к. `push_back` реализован как обеспечивающий строг.гар.безоп.искл. по стандарту(?) (хоть и не является `noexcept`).
2. Метод `pop` возвращает элемент по значению, а значит копирует, что потенциально может вызвать исключение
    - тогда элемент `tmp` будет утерян, соответственно обеспечивается только базовая гарантия безопасности исключений

Итого:

- Обеспечить строгую гарантию безопасности для метода pop, который возвращает выталкиваемый элемент, не получается из-за возможного исключения при копировании возвращаемого объекта.

- Если у класса, объекты которого хранятся в стеке, есть небросающий перемещающий конструктор, то исходная реализация обеспечивает строгую гарантию безопасности исключений.

```
T a;
a = b;      // оп.присваивания (перемещ. или нет не важно)
T a1 = b;   // к.копирования (перемещ. или нет не важно)
```

- Проблема обеспечения строгой гарантии безопасности возвращающего метода pop актуальна для произвольных контейнеров.

## <a id='toc3_8_'></a>[Проектирование с учётом исключений (продолжение)](#toc0_)

Будем возвращать по ссылке в аргументе.

Рассмотрим традиционный интерфейс стека:

    template<class T>
    struct Stack
    {
        void push(T const& t)
        {
            data_.push_back(t);
        }
        
        void pop(T & res)
        {
            res = data_.back();
            data_.pop_back();
        }
        
        std::vector<T> data_;
    };

Тогда `res` произойдет **присваивание** (копи или мув - не важно, главное - не установка указателя) и можно будет удалять элемент из стека.

Минус - поменялась сигнатура на неудобную.

## <a id='toc3_9_'></a>[Использование unique_ptr](#toc0_)

Будем возвращать элемент по указателю

    template<class T>
    struct Stack
    {
        void push(T const& t)
        {
            data_.push_back(t);
        }

        unique_ptr<T> pop()
        {
            % unique_ptr<T> tmp(new T(data_.back()));
            data_.pop_back();
            return std::move(tmp); // тут не надо мувить!!!!!
        }
        
        std::vector<T> data_;
    };

Создается в куче объект - элемент вершины стека, и оборачивается в умный указатель.

Вместо к.копирования для типа Т, кот. может бросить исключение, используется перемещ.конструктор умного указателя, который не бросает исключения.

Минус - получили неестественную сигнатуру для данного метода с уникальным указателем.

*) при возвращении локального объекта функции (каким и является `tmp`) по значению будет вызываться перемещающий конструктор. Плюс, без `std::move` компилятору позволяется сделать оптимизацию возвращаемого значения (RVO), и потенциально оно должно работать эффективнее

неполный список случаев, когда будут вызываться перемещающие методы:

- если передавать в них объект при помощи std::move();
- если передавать в них временный объект;
- если из функции по значению возвращается локальный объект функции.

**Проблема архитектуры**

Тут мы пытались в одном методе решить 2 задачи - вернуть вершину и удалить ее из стека, все получается коряво и неудобно, хоть и будет работать.

В STL аналогичный стек сделали из двух методов `pop()` - удаляет, `top()` - возвращает вершину, и это правильно!

## <a id='toc3_10_'></a>[Заключение](#toc0_)

- Проектируйте архитектуру приложения с учётом исключений.
- Функции, не бросающие исключения, нужно объявлять как noexcept.
- Все использующие исключения функции должны обеспечивать как минимум базовую гарантию безопасности исключений.
- Там, где это возможно, старайтесь обеспечить строгую гарантию безопасности исключений.
- Используйте swap, умные указатели и другие RAII объекты для обеспечения строгой безопасности исключений.